# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [318]:
# All these packages need to be installed from pip
import requests  # for http requests
import bs4  # called `beautifulsoup4`, an html parser
import pandas  # gives us DataFrames
import docx  # reading MS doc files, install as `python-docx`

# Stuff for pdfs
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

# These come with Python
import re  # for regexs
import urllib.parse  # For joining urls
import io  # for making http requests look like files
import json  # For Tumblr API responses
import os.path  # For checking if files exist
import os  # For making directories

In [319]:
bbc_fangfang_report = "https://www.bbc.com/"
bbc_fangfang_report = "https://www.bbc.com/"
bbc_fangfang_report = "https://www.bbc.com/news/world-asia-54987675"

In [320]:
bbc_content_request = requests.get(bbc_fangfang_report)
print(bbc_content_request)
print('---')
print(bbc_content_request.text[:1000])

<Response [200]>
---
<!DOCTYPE html><html lang="en-GB" class="no-js"><head><meta charSet="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1" /><title data-rh="true">Fang Fang: Author vilified for Wuhan Diary speaks out a year on - BBC News</title><meta data-rh="true" name="description" content="Wuhan writer Fang Fang talks about the price she has had to pay for her controversial lockdown diaries."/><meta data-rh="true" name="theme-color" content="#FFFFFF"/><meta data-rh="true" property="article:author" content="https://www.facebook.com/bbcnews"/><meta data-rh="true" property="article:section" content="Asia"/><meta data-rh="true" property="fb:admins" content="100004154058350"/><meta data-rh="true" property="fb:app_id" content="1609039196070050"/><meta data-rh="true" property="fb:pages" content="1143803202301544,317278538359186,1392506827668140,742734325867560,185246968166196,156060587793370,137920769558355,193435954068976,21263239760,156400551056385,92939969707

In [321]:
bbc_content_soup = bs4.BeautifulSoup(bbc_content_request.text, 'html.parser')
print(bbc_content_soup.text[:2000])

Fang Fang: Author vilified for Wuhan Diary speaks out a year on - BBC NewsBBC HomepageSkip to contentAccessibility HelpBBC AccountHomeNewsSportReelWorklifeTravelFutureCultureMenuMorecaret-downsearchSearchHomeNewsSportReelWorklifeTravelFutureCultureMusicTVWeatherSoundsnoClose menuBBC NewslistMenuHomeCoronavirusVideoWorldAsiaUKBusinessTechScienceStoriesEntertainment & ArtslistMoreHealthWorld News TVIn PicturesReality CheckNewsbeatLong ReadsAsiaChinaIndiaFang Fang: Author vilified for Wuhan Diary speaks out a year onBy Fan Wang and Grace TsoiBBC World Service, Hong KongPublishedduration1 day agoSharenocloseShare pagelinkCopy linkAbout sharingRelated TopicsCoronavirus pandemicimage copyrightWu BaojianShe has faced a nationalist backlash for her diaries documenting life in Wuhan in the early days of the coronavirus outbreak, but Chinese author Fang Fang she says she will not be silenced."When facing a catastrophe, it's vital to voice your opinion and give your advice," she told BBC Chinese 

In [322]:
# Inspect what Python receives. To do this we can save the html requests obtained.

bbc_save = 'news.html'
with open(bbc_save, mode='w', encoding='utf-8') as f:
    f.write(bbc_content_request.text)

In [323]:
content_p_tags = bbc_content_soup.body.findAll('p')
for p_tag in content_p_tags:
    print(p_tag.text)

By Fan Wang and Grace TsoiBBC World Service, Hong Kong
She has faced a nationalist backlash for her diaries documenting life in Wuhan in the early days of the coronavirus outbreak, but Chinese author Fang Fang she says she will not be silenced.
"When facing a catastrophe, it's vital to voice your opinion and give your advice," she told BBC Chinese in a rare email interview with international media.
In late January, when Wuhan became the first place in the world to enter a state of complete lockdown, many of the city's 11 million residents found solace in reading Fang Fang's online diaries. They also provided a revealing glimpse into the city where the virus first emerged.
The 65-year-old's daily posts on her Weibo account, the Chinese equivalent of Twitter, chronicled life living alone with her dog during the lockdown, as well as what she described as the dark side of the authority's response.
They were initially well-received, but later provoked a wave of criticism from those who saw 

In [324]:
# The content of this news is almost perfect, 
# so the cleaning work I'm doing is actually a littile bit superfluous.
# Anyway, the following code is used to meet the requirements of the homework

content_paragraphs = []
for p_tag in content_p_tags:
    content_paragraphs.append(re.sub(r'\[\d+\]', '', p_tag.text))

# Further cleaning the article
cleaned_content_paragraphs = []
for paragraph in content_paragraphs:
    cleaned_paragraph = re.sub(r'Â©', "", paragraph) # Remove non-readable characters
    cleaned_paragraph = re.sub(r' +', " ", cleaned_paragraph) # Remove extra spaces
    cleaned_paragraph = re.sub(r'^\s+', "", cleaned_paragraph) # Remove leading spaces
    cleaned_paragraph = re.sub(r'\s+$', "", cleaned_paragraph) # # Remove trailing spaces
    cleaned_paragraph = re.sub(r'"', "'", cleaned_paragraph) # To meet the homework's requirement
    cleaned_content_paragraphs.append(cleaned_paragraph)
    

content_paragraphs_df = pandas.DataFrame({'paragraph_text': cleaned_content_paragraphs})
print(content_paragraphs_df)

                                       paragraph_text
0   By Fan Wang and Grace TsoiBBC World Service, H...
1   She has faced a nationalist backlash for her d...
2   'When facing a catastrophe, it's vital to voic...
3   In late January, when Wuhan became the first p...
4   The 65-year-old's daily posts on her Weibo acc...
..                                                ...
68  Owners of Tiger King zoo ordered to surrender ...
69    US: China 'committed genocide against Uighurs'8
70  PM's aid cut damaged UK's moral leadership, sa...
71    Bali to deport influencer over Twitter thread10
72  2021 BBC. The BBC is not responsible for the c...

[73 rows x 1 columns]



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [325]:
print(content_p_tags)

[<p class="css-1pjc44v-Contributor e5xb54n0"><span><strong>By Fan Wang and Grace Tsoi</strong><br/>BBC World Service, Hong Kong</span></p>, <p><style data-emotion-css="14iz86j-BoldText">.css-14iz86j-BoldText{font-weight:bold;}</style><b class="css-14iz86j-BoldText e5tfeyi0">She has faced a nationalist backlash for her diaries documenting life in Wuhan in the early days of the coronavirus outbreak, but Chinese author Fang Fang she says she will not be silenced.</b></p>, <p>"When facing a catastrophe, it's vital to voice your opinion and give your advice," she told BBC Chinese in a rare email interview with international media.</p>, <p>In late January, when Wuhan became the first place in the world to enter a state of complete lockdown, many of the city's 11 million residents found solace in reading Fang Fang's online diaries. They also provided a revealing glimpse into the city where the virus first emerged.</p>, <p>The 65-year-old's daily posts on her Weibo account, the Chinese equival

In [351]:
# Notice here there are both relative links and absolute links.

bbc_base_url = "https://www.bbc.co.uk"
other_page_urls = []

for paragraph_number, p_tag in enumerate(content_p_tags):
    tag_links = p_tag.findAll('a', href=re.compile('^/news/'))

    # Deal with relative links first
    for a_tag in tag_links:
        relurl = a_tag.get('href')
        link_text = a_tag.text

        other_page_urls.append(
            (
                urllib.parse.urljoin(bbc_base_url, relurl),
                paragraph_number,
                link_text,
            )
        )

    # Deal with absolute links
    tag_links = p_tag.findAll('a', href=re.compile('https://www.bbc.co.uk'))
    for a_tag in tag_links:
        absolute_url = a_tag.get('href')
        link_text = a_tag.text
        other_page_urls.append(
            (absolute_url,
             paragraph_number,
             link_text,
             ))

print(other_page_urls[:5])
print('--------------------')
print(len(other_page_urls))

[('https://www.bbc.co.uk/news/world-55042935', 43, 'BBC 100 Women 2020: Who is on the list this year?'), ('https://www.bbc.co.uk/news/stories-55020994', 44, 'Sanna Marin: The feminist PM leading a coalition of women'), ('https://www.bbc.co.uk/news/world-us-canada-55729221', 45, 'Trump includes ex-aide Bannon in swathe of pardons'), ('https://www.bbc.co.uk/news/world-us-canada-55728852', 47, 'Biden to be sworn in as Trump leaves White House'), ('https://www.bbc.co.uk/news/world-us-canada-55723522', 48, "US: China 'committed genocide against Uighurs'")]
--------------------
26


In [327]:
content_paragraphs_df['source'] = [bbc_fangfang_report] * len(content_paragraphs_df['paragraph_text'])
print(len(content_paragraphs_df['paragraph_text']))
content_paragraphs_df['paragraph_number'] = range(len(content_paragraphs_df['paragraph_text']))
print(range(len(content_paragraphs_df['paragraph_text'])))
content_paragraphs_df

73
range(0, 73)


,paragraph_text,source,paragraph_number
0,"By Fan Wang and Grace TsoiBBC World Service, H...",https://www.bbc.com/news/world-asia-54987675,0
1,She has faced a nationalist backlash for her d...,https://www.bbc.com/news/world-asia-54987675,1
2,"'When facing a catastrophe, it's vital to voic...",https://www.bbc.com/news/world-asia-54987675,2
3,"In late January, when Wuhan became the first p...",https://www.bbc.com/news/world-asia-54987675,3
4,The 65-year-old's daily posts on her Weibo acc...,https://www.bbc.com/news/world-asia-54987675,4
...,...,...,...
68,Owners of Tiger King zoo ordered to surrender ...,https://www.bbc.com/news/world-asia-54987675,68
69,US: China 'committed genocide against Uighurs'8,https://www.bbc.com/news/world-asia-54987675,69
70,"PM's aid cut damaged UK's moral leadership, sa...",https://www.bbc.com/news/world-asia-54987675,70
71,Bali to deport influencer over Twitter thread10,https://www.bbc.com/news/world-asia-54987675,71


# <span style="color:red">Section 2: recursive</span>

In [328]:
content_paragraphs_df['source_paragraph_number'] = [
    None] * len(content_paragraphs_df['paragraph_text'])
content_paragraphs_df['source_paragraph_text'] = [
    None] * len(content_paragraphs_df['paragraph_text'])


def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    # Make a dict to store data before adding it to the DataFrame
    parsDict = {'source': [], 'paragraph_number': [], 'paragraph_text': [
    ], 'source_paragraph_number': [],  'source_paragraph_text': []}
    # Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    # enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        # same regex as before
        parsDict['paragraph_text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph_number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source_paragraph_number'].append(sourceParNum)
        parsDict['source_paragraph_text'].append(sourceText)

    # Generate new other_page_urls <--- type: a list of tuples
    global other_page_urls
    other_page_urls = []

    contentPTags = soup.body.findAll('p')
    for paragraphNum, pTag in enumerate(contentPTags):
        tagLinks = pTag.findAll('a', href=re.compile('^/news/'))
        for aTag in tagLinks:
            relurl = aTag.get('href')
            linkText = aTag.text
            other_page_urls.append((
                urllib.parse.urljoin(bbc_base_url, relurl),
                paragraphNum,
                linkText,
            ))

        tagLinks = pTag.findAll('a', href=re.compile('https://www.bbc.co.uk'))
        for aTag in tagLinks:
            absolute_url = aTag.get('href')
            linkText = aTag.text
            other_page_urls.append((
                absolute_url,
                paragraphNum,
                linkText,
            ))
    return pandas.DataFrame(parsDict)

In [353]:
end_depth_level = 3
while end_depth_level != 0:
    end_depth_level -= 1
    print("Current depth level:", end_depth_level)
    for url_tuple in other_page_urls[1:2]:
        content_paragraphs_df = content_paragraphs_df.append(getTextFromWikiPage(*url_tuple), ignore_index=True)
content_paragraphs_df

Current depth level: 2
Current depth level: 1
Current depth level: 0


,paragraph_text,source,paragraph_number,source_paragraph_number,source_paragraph_text
0,"By Fan Wang and Grace TsoiBBC World Service, H...",https://www.bbc.com/news/world-asia-54987675,0,None,None
1,She has faced a nationalist backlash for her d...,https://www.bbc.com/news/world-asia-54987675,1,None,None
2,"'When facing a catastrophe, it's vital to voic...",https://www.bbc.com/news/world-asia-54987675,2,None,None
3,"In late January, when Wuhan became the first p...",https://www.bbc.com/news/world-asia-54987675,3,None,None
4,The 65-year-old's daily posts on her Weibo acc...,https://www.bbc.com/news/world-asia-54987675,4,None,None
...,...,...,...,...,...
1940,Owners of Tiger King zoo ordered to surrender ...,https://www.bbc.co.uk/news/stories-55020994,97,44,Sanna Marin: The feminist PM leading a coaliti...
1941,US: China 'committed genocide against Uighurs'8,https://www.bbc.co.uk/news/stories-55020994,98,44,Sanna Marin: The feminist PM leading a coaliti...
1942,"PM's aid cut damaged UK's moral leadership, sa...",https://www.bbc.co.uk/news/stories-55020994,99,44,Sanna Marin: The feminist PM leading a coaliti...
1943,Bali to deport influencer over Twitter thread10,https://www.bbc.co.uk/news/stories-55020994,100,44,Sanna Marin: The feminist PM leading a coaliti...


In [330]:
# I only extract one link from "one" website. Therefore, 
# the following result shows that I extract info from "three" different sites

content_paragraphs_df.source.unique()

array(['https://www.bbc.com/news/world-asia-54987675',
       'https://www.bbc.co.uk/news/stories-55020994',
       'https://www.bbc.co.uk/news/world-us-canada-55729221',
       'https://www.bbc.co.uk/news/world-us-canada-44244687'],
      dtype=object)


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


## Raw Files

In [370]:
target_dir = "./HW_txt_data"
text_list = []
file_names = []

for file in (file for file in os.scandir(target_dir) if file.is_file() and not file.name.startswith(".")):
    with open(file.path, encoding='utf-8') as f:
        text_list.append(f.read())
    file_names.append(file.name)

txt_df = pandas.DataFrame({'text': text_list}, index=file_names)
txt_df

,text
test32112.txt,hehe\nhaha\n
sometextfile.txt,A line\nAnother line\nA line with a few unusua...
problem2.txt,"def lru_cache(func, maxsize=128):\n '''\n ..."
problem1.txt,### No importing other modules. You should are...
sample1.txt,Utilitatis causa amicitia est quaesita.\nLorem...
sample2.txt,"Aeque enim contingit omnibus fidibus, ut incon..."
sample3.txt,Quod equidem non reprehendo;\nLorem ipsum dolo...


In [355]:
pdf_link = "https://www.cs.cornell.edu/~cristian/Echoes_of_power_files/echoes_of_power.pdf"
info_extraction_request = requests.get(pdf_link, stream=True)
print(info_extraction_request.text[:500])

%PDF-1.5
%����
3 0 obj <<
/Length 4875      
/Filter /FlateDecode
>>
stream
xڍ[I��8��ׯ�95�bq_��v���j�]�pLT�� 	����,կ�����=��o���w��?�	�����o��&wa�qT�=���/�� ��"�{���꽭6a��Z�o�Q^x��$��n��{��_7�8I���9�
���-�����p@���C5��I��w��
v�E�=��4�~����E�z��e{�i��f�n�I�d��0+�6!.�Զ������D�o����x��ʕ�߫Я��|��Q,Q�յ�_�I�]{�����x�[�T�|�s<��>7.uRkm�8usg��~^


In [340]:
def readPDF(pdfFile):
    # Based on code from http://stackoverflow.com/a/20905381/4955164
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(
        rsrcmgr, retstr, laparams=layoutParams, codec=codec)
    # We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos = set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

In [341]:
info_extraction_bytes = io.BytesIO(info_extraction_request.content)

In [368]:
# Extract the PDF's title
pdf_title = [readPDF(info_extraction_bytes)[:15]]
print(pdf_title)

# Extract some of the PDF's texts and clean them
pdf_text = readPDF(info_extraction_bytes)[55:2500].split('\n')
cleaned_pdf_text = []
for item in pdf_text:
    if len(item) > 0:
        cleaned_pdf_text.append(re.sub(r'\n', '', item))

print(cleaned_pdf_text[:4])

['Echoes of Power']
['in Social Interaction', 'Cristian Danescu-Niculescu-Mizil', 'Cornell University', 'cristian@cs.cornell.edu,']


In [366]:
pdf_df = pandas.DataFrame({"title": pdf_title*len(cleaned_pdf_text)})
pdf_df['text'] = cleaned_pdf_text
print(pdf_df)

              title                                               text
0   Echoes of Power                              in Social Interaction
1   Echoes of Power                   Cristian Danescu-Niculescu-Mizil
2   Echoes of Power                                 Cornell University
3   Echoes of Power                           cristian@cs.cornell.edu,
4   Echoes of Power                               llee@cs.cornell.edu,
5   Echoes of Power                              bopang@yahoo-inc.com,
6   Echoes of Power                                        Lillian Lee
7   Echoes of Power                                 Cornell University
8   Echoes of Power                                            Bo Pang
9   Echoes of Power                                             Yahoo!
10  Echoes of Power                                      Jon Kleinberg
11  Echoes of Power                                 Cornell University
12  Echoes of Power                            kleinber@cs.cornell.edu
13  Ec

In [345]:
word_docx_link = "https://eprints.soton.ac.uk/417883/1/Claeys_et_al_LCAH_v3_15_December.docx"
word_docx_request = requests.get(word_docx_link, stream=True)
word_docx_content = docx.Document(io.BytesIO(word_docx_request.content))
for paragraph in word_docx_content.paragraphs[:3]:
    print(paragraph.text)

Prevention of vaccine-matched and mismatched influenza in children 6−35 months of age: a multinational randomized trial across five influenza seasons
Authors: Claeys C1, MD; Zaman K*2, PhD; Dbaibo G3, MD; Li P4, PhD; Izu A5, MS; Kosalaraksa P6, MD; Rivera L7, MD; Acosta B8, MD; Arroba Basanta ML9, MD; Aziz A2, MD; Cabanero MA10, PhD; Chandrashekaran V11, MS; Corsaro B11, PhD; Cousin L12, MD; Diaz A13, MD; Diez-Domingo J14, MD; Dinleyici EC15, MD; Faust SN16, MD; Friel D1, PhD; Garcia-Sicilia J17, MD; Gomez-Go GD18, MD; Gonzales MLA19, MS; Hughes SM20, MD; Jackowska T21, MD; Kant S22, MD; Lucero M23, MD; Malvaux L1, PhD; Mares Bermudez J24, MD; Martinon-Torres F25, PhD; Miranda M26, MD; Montellano M18, MD; Peix Sambola MA27, MD; Prymula R28, MD; Puthanakit T29, MD; Ruzkova R30, MD; Sadowska-Krawczenko I31, PhD; Salamanca de la Cueva I32, MD; Sokal E33, MD; Soni J34, MA; Szymanski H35, MD; Ulied A36, MD; Schuind A11, MD; Jain VKǂ37, MD; Innis BL38, MD; for the Flu4VEC Study Group 
* Co-f

In [371]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

In [372]:
word_docx_save = "word_docx_example.docx"
try:
    word_docx_content = docx.Document(downloadIfNeeded(word_docx_link, word_docx_save))
except Exception as e:
    print(e)

File is not a zip file


In [380]:
word_docx_content = docx.Document(downloadIfNeeded(word_docx_link, word_docx_save, mode='rb'))
word_docx_content_short_list = []

# Modify this to change the number of paragraphs retrived
for paragraph in word_docx_content.paragraphs[:10]:
    word_docx_content_short_list.append(paragraph.text[:10])

# Organize the info into a dataframe
word_docx_title = [word_docx_content_short_list[0]]
word_docx_short_content = [paragraph for paragraph in word_docx_content_short_list[1:] if len(paragraph) > 1]
word_docx_df_dict = {
    "Title": word_docx_title*len(word_docx_short_content),
    "Paragraph Content": word_docx_short_content
}
word_docx_df = pandas.DataFrame(word_docx_df_dict)
print(word_docx_df)

        Title Paragraph Content
0  Prevention        Authors: C
1  Prevention        * Co-first
2  Prevention        ǂ Co-last 
3  Prevention        Affiliatio
4  Prevention        Correspond
5  Prevention        Current wo
6  Prevention        Research i
